# Extracting YouTube Comments
Video ID : https://www.youtube.com/watch?v=hMtZfW2z9dw

In [1]:
import pandas as pd

**Original Comments**

In [2]:
# Scrape Comments for SQL Using Python Through The Youtube Data API
api_key =  # Insert your Api key here.

youtube = build('youtube', 'v3', developerKey=api_key)

box = [['name', 'comment', 'date', 'likes', 'reply_count', "parent_id", "is_parent"]]

code_lang = [{"id":"hMtZfW2z9dw"}]

for id_code in code_lang:
  data = youtube.commentThreads().list(part='snippet', videoId=id_code['id'], maxResults='100', textFormat="plainText").execute()
  for i in data["items"]:
    name = i["snippet"]['topLevelComment']["snippet"]["authorDisplayName"]
    comment = i["snippet"]['topLevelComment']["snippet"]["textDisplay"]
    published_at = i["snippet"]['topLevelComment']["snippet"]['publishedAt']
    likes = i["snippet"]['topLevelComment']["snippet"]['likeCount']
    parent = i["snippet"]['topLevelComment']["id"]
    totalReplyCount = i["snippet"]['totalReplyCount']
    
    box.append([name, comment, published_at, likes, totalReplyCount, parent, "Yes" ])          
                        
    while ("nextPageToken" in data):
      data = youtube.commentThreads().list(part='snippet', videoId=id_code['id'], pageToken=data["nextPageToken"],
                                             maxResults='100', textFormat="plainText").execute()
                                             
      for i in data["items"]:
        name = i["snippet"]['topLevelComment']["snippet"]["authorDisplayName"]
        comment = i["snippet"]['topLevelComment']["snippet"]["textDisplay"]
        published_at = i["snippet"]['topLevelComment']["snippet"]['publishedAt']
        likes = i["snippet"]['topLevelComment']["snippet"]['likeCount']
        totalReplyCount = i["snippet"]['totalReplyCount']
        parent = i["snippet"]['topLevelComment']["id"]

        box.append([name, comment, published_at, likes, totalReplyCount, parent, "Yes" ])
                

df = pd.DataFrame({'name': [i[0] for i in box], 'comment': [i[1] for i in box], 'published_at': [i[2] for i in box],
                       'likes': [i[3] for i in box], 'Reply Count': [i[4] for i in box], 'parent_id': [i[5] for i in box], 'is_parent': [i[6] for i in box]})      
sql_vids = pd.DataFrame([])
sql_vids = sql_vids.append(df, ignore_index = True)

In [8]:
sql_vids_parents = sql_vids.drop(0, axis=0)
sql_vids_parents['Reply Count'] = sql_vids_parents['Reply Count'].map(int)

In [12]:
# Filter only comments with replies
sql_vids_parents_replies = sql_vids_parents[sql_vids_parents["Reply Count"]>0]

In [14]:
# of comments with replies
print(len(sql_vids_parents_replies))
sql_vids_parents_replies.head()

1414


,name,comment,published_at,likes,Reply Count,parent_id,is_parent
1,schmoyoho,Stream the track! https://open.spotify.com/alb...,2018-07-12T18:42:05Z,13950,188,UgywoWu8TaqkOzGi8HF4AaABAg,Yes
15,Gorsborshmn,I watch this before every episode of One Piece.,2022-04-18T03:26:38Z,0,1,UgzZ6oGZmPH4Q9bJCGp4AaABAg,Yes
20,Steven VG Fernandez,April 16 2022..,2022-04-15T23:49:18Z,2,1,UgwQ_-KChqEU20m7iT54AaABAg,Yes
24,Matt John,Whose here in 2022!?,2022-04-14T20:16:56Z,2,1,Ugwhyc1DC7HAEe9dSzx4AaABAg,Yes
52,Miranda Fox,1 ❤️ Hermosa eleccion Likeex.Uno de mejor \n2 ...,2022-04-07T12:59:08Z,15,1,UgzP9gOKEwNWPfs6zxF4AaABAg,Yes


**Replies only**

In [21]:
api_key = "" # Insert your Api key here.

youtube = build('youtube', 'v3', developerKey=api_key)

box = [['name', 'comment', 'date', 'likes', 'reply_count', "parent_id", "is_parent"]]

for parent_id in sql_vids_parents_replies['parent_id']:
  data = youtube.comments().list(part='snippet', maxResults='100', textFormat="plainText", parentId=parent_id).execute()
  for i in data["items"]:
    name = i["snippet"]["authorDisplayName"]
    comment = i["snippet"]["textDisplay"]
    published_at = i["snippet"]['publishedAt']
    likes = i["snippet"]['likeCount']
    parent = parent_id
    totalReplyCount = ''
    
    box.append([name, comment, published_at, likes, totalReplyCount, parent, "No" ])          
                        
    while ("nextPageToken" in data):
      data = youtube.comments().list(part='snippet', pageToken=data["nextPageToken"],
                                             maxResults='100', textFormat="plainText", parentId=parent_id).execute()
                                             
      for i in data["items"]:
        name = i["snippet"]["authorDisplayName"]
        comment = i["snippet"]["textDisplay"]
        published_at = i["snippet"]['publishedAt']
        likes = i["snippet"]['likeCount']
        parent = parent_id
        totalReplyCount = ''

        box.append([name, comment, published_at, likes, totalReplyCount, parent, "No" ])
                

df_replies = pd.DataFrame({'name': [i[0] for i in box], 'comment': [i[1] for i in box], 'published_at': [i[2] for i in box],
                       'likes': [i[3] for i in box], 'Reply Count': [i[4] for i in box], 'parent_id': [i[5] for i in box], 'is_parent': [i[6] for i in box]})      
sql_vids_replies = pd.DataFrame([])
sql_vids_replies = sql_vids_replies.append(df_replies, ignore_index = True)

In [27]:
sql_vids_replies.drop(0, axis=0, inplace=True)

In [29]:
sql_vids_replies.head()

,name,comment,published_at,likes,Reply Count,parent_id,is_parent
1,Lillie Britt,This go hard and it's been 10yrs already lolz ...,2022-04-23T04:09:14Z,0,,UgywoWu8TaqkOzGi8HF4AaABAg,No
2,Brazilian Simon,Lmao,2018-12-17T06:31:21Z,0,,UgywoWu8TaqkOzGi8HF4AaABAg,No
3,ChocoMintTea,Lmao,2018-12-11T19:52:36Z,1,,UgywoWu8TaqkOzGi8HF4AaABAg,No
4,The Pixel Queen,This will shape the history of YouTube,2018-12-05T21:30:19Z,0,,UgywoWu8TaqkOzGi8HF4AaABAg,No
5,Ringtonez Drashopen,You must be joking! Get the track😂?,2018-12-04T11:01:18Z,0,,UgywoWu8TaqkOzGi8HF4AaABAg,No


**Combining the datasets**

In [30]:
df_all = pd.concat([sql_vids, sql_vids_replies])

In [31]:
# Total # of records
len(df_all)

136584

In [33]:
# Saving files
df_all.to_csv('/content/drive/MyDrive/3. IU Courses/Courses/7. FADS/05202022_youtube_comments.csv',encoding='utf_8_sig')
df_all.to_pickle('/content/drive/MyDrive/3. IU Courses/Courses/7. FADS/05202022_youtube_comments.pkl')